In [36]:
import numpy as np
from numpy.core.fromnumeric import _all_dispatcher
import pandas as pd
import joblib
np.random.seed(2021)

In [37]:
##########################################################
######### training code (without any validation) #########

# load data
print('loading train.gz...')
# use only a subset of rows - you should use all rows eventually
df_train = pd.read_csv("train.gz", compression='gzip', nrows=20000, header='infer')
Y = df_train['click']
# discard some columns
unused_cols = ["id", 'site_id', 'app_id']
df_train.drop(unused_cols, axis=1, inplace=True)

loading train.gz...


In [38]:
# create a copy to prevent from modifying the original dataset
df_copy = df_train.copy()

In [39]:
# nunique() shows that the three columns are same across 
df_copy.drop(['hour', 'app_domain', 'app_category', 'click'], axis=1,inplace=True)

In [40]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(df_copy, Y, test_size=0.3)

In [41]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14000 entries, 10657 to 6201
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   C1                14000 non-null  int64 
 1   banner_pos        14000 non-null  int64 
 2   site_domain       14000 non-null  object
 3   site_category     14000 non-null  object
 4   device_id         14000 non-null  object
 5   device_ip         14000 non-null  object
 6   device_model      14000 non-null  object
 7   device_type       14000 non-null  int64 
 8   device_conn_type  14000 non-null  int64 
 9   C14               14000 non-null  int64 
 10  C15               14000 non-null  int64 
 11  C16               14000 non-null  int64 
 12  C17               14000 non-null  int64 
 13  C18               14000 non-null  int64 
 14  C19               14000 non-null  int64 
 15  C20               14000 non-null  int64 
 16  C21               14000 non-null  int64 
dtypes: int64(

In [7]:
import category_encoders as ce
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

# one_hot_features = ['C1', 'device_type', 'device_conn_type', 'C18']
one_hot_features = [0,7,8,13]
one_hot_transformer = OneHotEncoder(drop='first')

# target_features = ['banner_pos','site_domain','site_category','device_id','device_ip','device_model','C14',
#                    'C15','C16','C17','C19','C20','C21']
target_features = [1,2,3,4,5,6,9,10,11,12,14,15,16]
target_transformer = ce.JamesSteinEncoder()
        
preprocessor = ColumnTransformer(
    transformers=[
        ('one_hot', one_hot_transformer, one_hot_features),
        ('target', target_transformer, target_features)])

In [8]:
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostClassifier(iterations=20,learning_rate=0.1,depth=7, eval_metric='Logloss'))])

clf.fit(X_train.values, y_train.values.reshape(-1,1))
y_pred = clf.predict_proba(X_test.values)[:, 1]
print("model logloss: %.3f" % log_loss(y_test, y_pred))

/Users/williamlee/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0:	learn: 0.6160399	total: 67.6ms	remaining: 1.28s
1:	learn: 0.5533086	total: 72.8ms	remaining: 655ms
2:	learn: 0.5003023	total: 78.4ms	remaining: 444ms
3:	learn: 0.4554998	total: 82.6ms	remaining: 330ms
4:	learn: 0.4170045	total: 86.5ms	remaining: 260ms
5:	learn: 0.3836853	total: 90ms	remaining: 210ms
6:	learn: 0.3553075	total: 92.8ms	remaining: 172ms
7:	learn: 0.3301982	total: 95.4ms	remaining: 143ms
8:	learn: 0.3088449	total: 98.5ms	remaining: 120ms
9:	learn: 0.2898885	total: 101ms	remaining: 101ms
10:	learn: 0.2740571	total: 104ms	remaining: 85ms
11:	learn: 0.2594245	total: 107ms	remaining: 71.1ms
12:	learn: 0.2470110	total: 109ms	remaining: 58.8ms
13:	learn: 0.2355591	total: 112ms	remaining: 48ms
14:	learn: 0.2252611	total: 115ms	remaining: 38.2ms
15:	learn: 0.2156349	total: 117ms	remaining: 29.3ms
16:	learn: 0.2068159	total: 120ms	remaining: 21.2ms
17:	learn: 0.2003635	total: 124ms	remaining: 13.7ms
18:	learn: 0.1942897	total: 126ms	remaining: 6.64ms
19:	learn: 0.1878198	total: 1

#### First baseline model has a very high logloss at 0.525.

In [9]:
categorical_f = ['C1','device_type','device_conn_type','C18','banner_pos','site_domain','site_category',
                 'device_id','device_ip','device_model','C14','C15','C16','C17','C19','C20','C21']
cat = CatBoostClassifier(iterations=20,learning_rate=0.1,depth=7, eval_metric='Logloss')
cat.fit(X_train, y_train,cat_features=categorical_f)
y_pred_cat = cat.predict_proba(X_test.values)[:, 1]
print("model logloss: %.3f" % log_loss(y_test, y_pred_cat))

0:	learn: 0.6538872	total: 10.9ms	remaining: 207ms
1:	learn: 0.6239209	total: 17ms	remaining: 153ms
2:	learn: 0.5968163	total: 24.5ms	remaining: 139ms
3:	learn: 0.5743491	total: 32.3ms	remaining: 129ms
4:	learn: 0.5560601	total: 37.1ms	remaining: 111ms
5:	learn: 0.5410683	total: 42ms	remaining: 98ms
6:	learn: 0.5274857	total: 49.6ms	remaining: 92.2ms
7:	learn: 0.5156095	total: 57.6ms	remaining: 86.4ms
8:	learn: 0.5057243	total: 61.6ms	remaining: 75.3ms
9:	learn: 0.4972880	total: 68ms	remaining: 68ms
10:	learn: 0.4904956	total: 73.2ms	remaining: 59.9ms
11:	learn: 0.4842027	total: 79.5ms	remaining: 53ms
12:	learn: 0.4787759	total: 87.1ms	remaining: 46.9ms
13:	learn: 0.4747286	total: 91.5ms	remaining: 39.2ms
14:	learn: 0.4708115	total: 98.2ms	remaining: 32.7ms
15:	learn: 0.4677292	total: 105ms	remaining: 26.3ms
16:	learn: 0.4646293	total: 112ms	remaining: 19.7ms
17:	learn: 0.4629520	total: 115ms	remaining: 12.8ms
18:	learn: 0.4606878	total: 121ms	remaining: 6.37ms
19:	learn: 0.4586883	tot

#### using the default catboost encoder seems to get much better result

In [ ]:
# need to balance the data
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, RandomOverSampler
# smt = SMOTE()
ros = RandomOverSampler(random_state=0)
X_train_re, y_train_re = ros.fit_resample(X_train, y_train)

### Bayesian Search CV for hyperparameter tuning

In [31]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer

cat1 = CatBoostClassifier(iterations=20,learning_rate=0.1,depth=7,loss_function='Logloss', 
                          cat_features=categorical_f,verbose=False)

param = {
    'iterations': Integer(10, 1000),
    'depth': Integer(1, 8),
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),
    'random_strength': Real(1e-9, 10, 'log-uniform'),
    'bagging_temperature': Real(0.0, 1.0),
    'border_count': Integer(1, 255),
    'l2_leaf_reg': Integer(2, 30),
    'scale_pos_weight':Real(0.01, 1.0, 'uniform')
}

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
    cat1,
    param,
    scoring = LogLoss,
    n_iter=64,
    cv=5,
    random_state=42
)

# executes bayesian optimization
opt.fit(X_train, y_train)

BayesSearchCV(cv=5, error_score='raise',
              estimator=<catboost.core.CatBoostClassifier object at 0x7ff8930bd3d0>,
              fit_params=None, iid=True, n_iter=64, n_jobs=1, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=42,
              refit=True, return_train_score=False,
              scoring=make_scorer(log_loss, greater_is_better=False, needs_proba=True),
              search_spaces={'bagg...
                             'iterations': Integer(low=10, high=1000, prior='uniform', transform='identity'),
                             'l2_leaf_reg': Integer(low=2, high=30, prior='uniform', transform='identity'),
                             'learning_rate': Real(low=0.01, high=1.0, prior='log-uniform', transform='identity'),
                             'random_strength': Real(low=1e-09, high=10, prior='log-uniform', transform='identity'),
                             'scale_pos_weight': Real(low=0.01, high=1.0, prior='uniform', tr

In [35]:
opt.best_params_

OrderedDict([('bagging_temperature', 0.0),
             ('border_count', 48),
             ('depth', 8),
             ('iterations', 759),
             ('l2_leaf_reg', 3),
             ('learning_rate', 0.01),
             ('random_strength', 1e-09),
             ('scale_pos_weight', 1.0)])

In [20]:
from sklearn.metrics import make_scorer
LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

best_params={'bagging_temperature': 0.0,
 'border_count': 48,
 'depth': 8,
 'iterations': 759,
 'l2_leaf_reg': 3,
 'learning_rate': 0.01,
 'random_strength': 1e-09,
 'scale_pos_weight': 1.0}

best_params['iterations'] = 1000

In [21]:
opt_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostClassifier(**best_params,od_type='Iter', loss_function='Logloss'))])

from sklearn.model_selection import cross_val_score
log_ = cross_val_score(estimator = opt_clf, X = X_train_re.values, y = y_train_re.values.reshape(-1,1), 
                             cv = 5, scoring=LogLoss)

/Users/williamlee/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0:	learn: 0.6699473	total: 12.7ms	remaining: 12.7s
1:	learn: 0.6445387	total: 21.3ms	remaining: 10.6s
2:	learn: 0.6223039	total: 30.7ms	remaining: 10.2s
3:	learn: 0.6022946	total: 39.8ms	remaining: 9.91s
4:	learn: 0.5817503	total: 47.6ms	remaining: 9.47s
5:	learn: 0.5638663	total: 56.4ms	remaining: 9.34s
6:	learn: 0.5469955	total: 65ms	remaining: 9.22s
7:	learn: 0.5313910	total: 74.2ms	remaining: 9.21s
8:	learn: 0.5133033	total: 82.8ms	remaining: 9.12s
9:	learn: 0.4987086	total: 91.9ms	remaining: 9.1s
10:	learn: 0.4834985	total: 100ms	remaining: 9.01s
11:	learn: 0.4719541	total: 109ms	remaining: 8.99s
12:	learn: 0.4609683	total: 117ms	remaining: 8.92s
13:	learn: 0.4510272	total: 126ms	remaining: 8.87s
14:	learn: 0.4392642	total: 134ms	remaining: 8.82s
15:	learn: 0.4264733	total: 143ms	remaining: 8.81s
16:	learn: 0.4166240	total: 152ms	remaining: 8.8s
17:	learn: 0.4084694	total: 161ms	remaining: 8.8s
18:	learn: 0.3983935	total: 171ms	remaining: 8.83s
19:	learn: 0.3905817	total: 181ms	re

178:	learn: 0.1484217	total: 1.84s	remaining: 8.45s
179:	learn: 0.1483198	total: 1.86s	remaining: 8.46s
180:	learn: 0.1482203	total: 1.87s	remaining: 8.46s
181:	learn: 0.1480580	total: 1.89s	remaining: 8.47s
182:	learn: 0.1479635	total: 1.9s	remaining: 8.49s
183:	learn: 0.1478706	total: 1.92s	remaining: 8.5s
184:	learn: 0.1477798	total: 1.93s	remaining: 8.5s
185:	learn: 0.1476228	total: 1.95s	remaining: 8.54s
186:	learn: 0.1475258	total: 1.97s	remaining: 8.57s
187:	learn: 0.1474348	total: 1.99s	remaining: 8.57s
188:	learn: 0.1472853	total: 2s	remaining: 8.59s
189:	learn: 0.1472097	total: 2.02s	remaining: 8.61s
190:	learn: 0.1471174	total: 2.03s	remaining: 8.62s
191:	learn: 0.1469752	total: 2.05s	remaining: 8.63s
192:	learn: 0.1468523	total: 2.07s	remaining: 8.64s
193:	learn: 0.1467647	total: 2.08s	remaining: 8.64s
194:	learn: 0.1466843	total: 2.09s	remaining: 8.64s
195:	learn: 0.1466003	total: 2.1s	remaining: 8.64s
196:	learn: 0.1465160	total: 2.12s	remaining: 8.64s
197:	learn: 0.14639

346:	learn: 0.1380708	total: 3.92s	remaining: 7.38s
347:	learn: 0.1380226	total: 3.94s	remaining: 7.38s
348:	learn: 0.1379906	total: 3.96s	remaining: 7.38s
349:	learn: 0.1379317	total: 3.98s	remaining: 7.38s
350:	learn: 0.1378898	total: 3.99s	remaining: 7.38s
351:	learn: 0.1378635	total: 4.01s	remaining: 7.37s
352:	learn: 0.1378335	total: 4.02s	remaining: 7.37s
353:	learn: 0.1377769	total: 4.04s	remaining: 7.37s
354:	learn: 0.1377410	total: 4.05s	remaining: 7.36s
355:	learn: 0.1376879	total: 4.07s	remaining: 7.36s
356:	learn: 0.1376586	total: 4.08s	remaining: 7.35s
357:	learn: 0.1376240	total: 4.09s	remaining: 7.33s
358:	learn: 0.1375635	total: 4.1s	remaining: 7.32s
359:	learn: 0.1375114	total: 4.12s	remaining: 7.32s
360:	learn: 0.1374562	total: 4.13s	remaining: 7.31s
361:	learn: 0.1374062	total: 4.14s	remaining: 7.3s
362:	learn: 0.1373859	total: 4.15s	remaining: 7.28s
363:	learn: 0.1373299	total: 4.16s	remaining: 7.27s
364:	learn: 0.1372817	total: 4.17s	remaining: 7.26s
365:	learn: 0.

508:	learn: 0.1315967	total: 5.75s	remaining: 5.55s
509:	learn: 0.1315680	total: 5.77s	remaining: 5.54s
510:	learn: 0.1315311	total: 5.78s	remaining: 5.53s
511:	learn: 0.1314959	total: 5.79s	remaining: 5.52s
512:	learn: 0.1314398	total: 5.8s	remaining: 5.5s
513:	learn: 0.1313908	total: 5.81s	remaining: 5.49s
514:	learn: 0.1313595	total: 5.82s	remaining: 5.48s
515:	learn: 0.1313353	total: 5.83s	remaining: 5.47s
516:	learn: 0.1312993	total: 5.84s	remaining: 5.45s
517:	learn: 0.1312639	total: 5.85s	remaining: 5.44s
518:	learn: 0.1312171	total: 5.86s	remaining: 5.43s
519:	learn: 0.1311982	total: 5.87s	remaining: 5.42s
520:	learn: 0.1311733	total: 5.88s	remaining: 5.4s
521:	learn: 0.1311497	total: 5.89s	remaining: 5.39s
522:	learn: 0.1311252	total: 5.9s	remaining: 5.38s
523:	learn: 0.1311013	total: 5.91s	remaining: 5.37s
524:	learn: 0.1310380	total: 5.92s	remaining: 5.35s
525:	learn: 0.1310065	total: 5.93s	remaining: 5.34s
526:	learn: 0.1309762	total: 5.94s	remaining: 5.33s
527:	learn: 0.13

...

170:	learn: 0.1533698	total: 1.62s	remaining: 7.87s
171:	learn: 0.1532638	total: 1.63s	remaining: 7.86s
172:	learn: 0.1531685	total: 1.64s	remaining: 7.85s
173:	learn: 0.1530657	total: 1.65s	remaining: 7.84s
174:	learn: 0.1529648	total: 1.66s	remaining: 7.83s
175:	learn: 0.1528845	total: 1.67s	remaining: 7.82s
176:	learn: 0.1527851	total: 1.68s	remaining: 7.81s
177:	learn: 0.1526982	total: 1.69s	remaining: 7.8s
178:	learn: 0.1526213	total: 1.7s	remaining: 7.78s
179:	learn: 0.1525250	total: 1.71s	remaining: 7.78s
180:	learn: 0.1524416	total: 1.72s	remaining: 7.77s
181:	learn: 0.1523481	total: 1.73s	remaining: 7.76s
182:	learn: 0.1521967	total: 1.74s	remaining: 7.75s
183:	learn: 0.1521125	total: 1.74s	remaining: 7.74s
184:	learn: 0.1519110	total: 1.75s	remaining: 7.73s
185:	learn: 0.1518217	total: 1.76s	remaining: 7.72s
186:	learn: 0.1517444	total: 1.77s	remaining: 7.71s
187:	learn: 0.1516576	total: 1.78s	remaining: 7.7s
188:	learn: 0.1515204	total: 1.79s	remaining: 7.69s
189:	learn: 0.1

341:	learn: 0.1414576	total: 3.25s	remaining: 6.24s
342:	learn: 0.1414236	total: 3.26s	remaining: 6.24s
343:	learn: 0.1413757	total: 3.27s	remaining: 6.23s
344:	learn: 0.1413132	total: 3.28s	remaining: 6.22s
345:	learn: 0.1412686	total: 3.29s	remaining: 6.21s
346:	learn: 0.1412236	total: 3.29s	remaining: 6.2s
347:	learn: 0.1411584	total: 3.3s	remaining: 6.19s
348:	learn: 0.1411280	total: 3.31s	remaining: 6.18s
349:	learn: 0.1410757	total: 3.32s	remaining: 6.17s
350:	learn: 0.1410288	total: 3.33s	remaining: 6.16s
351:	learn: 0.1410002	total: 3.34s	remaining: 6.15s
352:	learn: 0.1409619	total: 3.35s	remaining: 6.14s
353:	learn: 0.1409094	total: 3.36s	remaining: 6.13s
354:	learn: 0.1408457	total: 3.37s	remaining: 6.12s
355:	learn: 0.1408144	total: 3.38s	remaining: 6.11s
356:	learn: 0.1407630	total: 3.38s	remaining: 6.1s
357:	learn: 0.1407006	total: 3.39s	remaining: 6.09s
358:	learn: 0.1406730	total: 3.4s	remaining: 6.08s
359:	learn: 0.1406299	total: 3.41s	remaining: 6.07s
360:	learn: 0.14

503:	learn: 0.1352483	total: 4.87s	remaining: 4.8s
504:	learn: 0.1352020	total: 4.88s	remaining: 4.79s
505:	learn: 0.1351528	total: 4.89s	remaining: 4.78s
506:	learn: 0.1351077	total: 4.9s	remaining: 4.77s
507:	learn: 0.1350924	total: 4.91s	remaining: 4.76s
508:	learn: 0.1350654	total: 4.92s	remaining: 4.75s
509:	learn: 0.1350418	total: 4.93s	remaining: 4.74s
510:	learn: 0.1349980	total: 4.94s	remaining: 4.72s
511:	learn: 0.1349563	total: 4.95s	remaining: 4.72s
512:	learn: 0.1349258	total: 4.96s	remaining: 4.71s
513:	learn: 0.1349025	total: 4.97s	remaining: 4.7s
514:	learn: 0.1348639	total: 4.98s	remaining: 4.69s
515:	learn: 0.1348411	total: 4.99s	remaining: 4.68s
516:	learn: 0.1348127	total: 4.99s	remaining: 4.67s
517:	learn: 0.1347558	total: 5s	remaining: 4.66s
518:	learn: 0.1347208	total: 5.01s	remaining: 4.65s
519:	learn: 0.1346766	total: 5.02s	remaining: 4.64s
520:	learn: 0.1346536	total: 5.03s	remaining: 4.63s
521:	learn: 0.1346280	total: 5.04s	remaining: 4.62s
522:	learn: 0.1345

673:	learn: 0.1301977	total: 6.47s	remaining: 3.13s
674:	learn: 0.1301928	total: 6.48s	remaining: 3.12s
675:	learn: 0.1301645	total: 6.49s	remaining: 3.11s
676:	learn: 0.1301296	total: 6.5s	remaining: 3.1s
677:	learn: 0.1301027	total: 6.5s	remaining: 3.09s
678:	learn: 0.1300621	total: 6.51s	remaining: 3.08s
679:	learn: 0.1300557	total: 6.52s	remaining: 3.07s
680:	learn: 0.1300275	total: 6.53s	remaining: 3.06s
681:	learn: 0.1300227	total: 6.54s	remaining: 3.05s
682:	learn: 0.1299900	total: 6.55s	remaining: 3.04s
683:	learn: 0.1299646	total: 6.56s	remaining: 3.03s
684:	learn: 0.1299439	total: 6.57s	remaining: 3.02s
685:	learn: 0.1299073	total: 6.58s	remaining: 3.01s
686:	learn: 0.1298810	total: 6.59s	remaining: 3s
687:	learn: 0.1298667	total: 6.6s	remaining: 2.99s
688:	learn: 0.1298395	total: 6.61s	remaining: 2.98s
689:	learn: 0.1298348	total: 6.62s	remaining: 2.97s
690:	learn: 0.1298065	total: 6.62s	remaining: 2.96s
691:	learn: 0.1297724	total: 6.63s	remaining: 2.95s
692:	learn: 0.12973

842:	learn: 0.1263122	total: 8.08s	remaining: 1.5s
843:	learn: 0.1262849	total: 8.09s	remaining: 1.5s
844:	learn: 0.1262748	total: 8.1s	remaining: 1.49s
845:	learn: 0.1262672	total: 8.11s	remaining: 1.48s
846:	learn: 0.1262348	total: 8.12s	remaining: 1.47s
847:	learn: 0.1262112	total: 8.13s	remaining: 1.46s
848:	learn: 0.1261829	total: 8.14s	remaining: 1.45s
849:	learn: 0.1261592	total: 8.15s	remaining: 1.44s
850:	learn: 0.1261241	total: 8.16s	remaining: 1.43s
851:	learn: 0.1261019	total: 8.17s	remaining: 1.42s
852:	learn: 0.1260935	total: 8.18s	remaining: 1.41s
853:	learn: 0.1260644	total: 8.19s	remaining: 1.4s
854:	learn: 0.1260305	total: 8.2s	remaining: 1.39s
855:	learn: 0.1260239	total: 8.2s	remaining: 1.38s
856:	learn: 0.1260008	total: 8.21s	remaining: 1.37s
857:	learn: 0.1259566	total: 8.22s	remaining: 1.36s
858:	learn: 0.1259233	total: 8.24s	remaining: 1.35s
859:	learn: 0.1258959	total: 8.24s	remaining: 1.34s
860:	learn: 0.1258879	total: 8.26s	remaining: 1.33s
861:	learn: 0.1258

In [22]:
print("Log loss: {:.3f}".format(-log_.mean()))

Log loss: 0.402


In [30]:
opt_clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', CatBoostClassifier(**best_params,od_type='Iter', eval_metric='Logloss'))])

opt_clf.fit(X_train.values, y_train.values.reshape(-1,1))
y_pred = opt_clf.predict_proba(X_test.values)[:, 1]
print("model logloss: %.3f" % log_loss(y_test, y_pred))

/Users/williamlee/opt/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


0:	learn: 0.6697220	total: 10.3ms	remaining: 10.3s
1:	learn: 0.6461048	total: 18ms	remaining: 8.96s
2:	learn: 0.6253606	total: 27.6ms	remaining: 9.18s
3:	learn: 0.6026194	total: 42.2ms	remaining: 10.5s
4:	learn: 0.5823713	total: 53.2ms	remaining: 10.6s
5:	learn: 0.5646853	total: 69.9ms	remaining: 11.6s
6:	learn: 0.5427890	total: 84.4ms	remaining: 12s
7:	learn: 0.5263117	total: 93ms	remaining: 11.5s
8:	learn: 0.5050964	total: 101ms	remaining: 11.2s
9:	learn: 0.4910259	total: 110ms	remaining: 10.9s
10:	learn: 0.4739767	total: 119ms	remaining: 10.7s
11:	learn: 0.4613058	total: 127ms	remaining: 10.5s
12:	learn: 0.4496900	total: 137ms	remaining: 10.4s
13:	learn: 0.4379222	total: 145ms	remaining: 10.2s
14:	learn: 0.4200535	total: 153ms	remaining: 10s
15:	learn: 0.4075263	total: 165ms	remaining: 10.1s
16:	learn: 0.3957930	total: 173ms	remaining: 9.98s
17:	learn: 0.3844064	total: 183ms	remaining: 10s
18:	learn: 0.3731632	total: 192ms	remaining: 9.94s
19:	learn: 0.3631718	total: 201ms	remaining

173:	learn: 0.1262526	total: 1.81s	remaining: 8.59s
174:	learn: 0.1261801	total: 1.82s	remaining: 8.6s
175:	learn: 0.1260545	total: 1.83s	remaining: 8.59s
176:	learn: 0.1259311	total: 1.85s	remaining: 8.6s
177:	learn: 0.1258206	total: 1.86s	remaining: 8.58s
178:	learn: 0.1257029	total: 1.87s	remaining: 8.59s
179:	learn: 0.1256298	total: 1.88s	remaining: 8.58s
180:	learn: 0.1255163	total: 1.9s	remaining: 8.58s
181:	learn: 0.1254068	total: 1.91s	remaining: 8.58s
182:	learn: 0.1253377	total: 1.93s	remaining: 8.63s
183:	learn: 0.1252727	total: 1.95s	remaining: 8.65s
184:	learn: 0.1251602	total: 1.97s	remaining: 8.68s
185:	learn: 0.1250912	total: 1.98s	remaining: 8.67s
186:	learn: 0.1250291	total: 1.99s	remaining: 8.65s
187:	learn: 0.1249216	total: 2s	remaining: 8.64s
188:	learn: 0.1248557	total: 2.01s	remaining: 8.65s
189:	learn: 0.1247964	total: 2.03s	remaining: 8.65s
190:	learn: 0.1246988	total: 2.04s	remaining: 8.63s
191:	learn: 0.1246358	total: 2.04s	remaining: 8.61s
192:	learn: 0.1245

339:	learn: 0.1175291	total: 3.64s	remaining: 7.07s
340:	learn: 0.1175038	total: 3.66s	remaining: 7.07s
341:	learn: 0.1174815	total: 3.67s	remaining: 7.06s
342:	learn: 0.1174488	total: 3.68s	remaining: 7.05s
343:	learn: 0.1174256	total: 3.69s	remaining: 7.03s
344:	learn: 0.1173906	total: 3.7s	remaining: 7.03s
345:	learn: 0.1173666	total: 3.71s	remaining: 7.01s
346:	learn: 0.1173308	total: 3.72s	remaining: 7s
347:	learn: 0.1172887	total: 3.73s	remaining: 6.99s
348:	learn: 0.1172738	total: 3.74s	remaining: 6.98s
349:	learn: 0.1172455	total: 3.75s	remaining: 6.97s
350:	learn: 0.1172042	total: 3.77s	remaining: 6.96s
351:	learn: 0.1171765	total: 3.77s	remaining: 6.95s
352:	learn: 0.1171601	total: 3.79s	remaining: 6.94s
353:	learn: 0.1171297	total: 3.8s	remaining: 6.93s
354:	learn: 0.1171151	total: 3.81s	remaining: 6.91s
355:	learn: 0.1170924	total: 3.82s	remaining: 6.91s
356:	learn: 0.1170646	total: 3.83s	remaining: 6.89s
357:	learn: 0.1170378	total: 3.84s	remaining: 6.88s
358:	learn: 0.117

515:	learn: 0.1139063	total: 5.46s	remaining: 5.13s
516:	learn: 0.1138838	total: 5.47s	remaining: 5.12s
517:	learn: 0.1138556	total: 5.49s	remaining: 5.11s
518:	learn: 0.1138414	total: 5.5s	remaining: 5.09s
519:	learn: 0.1138099	total: 5.51s	remaining: 5.08s
520:	learn: 0.1137878	total: 5.52s	remaining: 5.07s
521:	learn: 0.1137657	total: 5.53s	remaining: 5.06s
522:	learn: 0.1137566	total: 5.54s	remaining: 5.05s
523:	learn: 0.1137266	total: 5.55s	remaining: 5.04s
524:	learn: 0.1137094	total: 5.56s	remaining: 5.03s
525:	learn: 0.1136837	total: 5.57s	remaining: 5.02s
526:	learn: 0.1136729	total: 5.58s	remaining: 5.01s
527:	learn: 0.1136464	total: 5.59s	remaining: 5s
528:	learn: 0.1136189	total: 5.6s	remaining: 4.99s
529:	learn: 0.1136009	total: 5.61s	remaining: 4.98s
530:	learn: 0.1135793	total: 5.62s	remaining: 4.97s
531:	learn: 0.1135629	total: 5.63s	remaining: 4.96s
532:	learn: 0.1135540	total: 5.64s	remaining: 4.95s
533:	learn: 0.1135357	total: 5.66s	remaining: 4.93s
534:	learn: 0.113

689:	learn: 0.1105493	total: 7.29s	remaining: 3.27s
690:	learn: 0.1105369	total: 7.3s	remaining: 3.26s
691:	learn: 0.1105068	total: 7.3s	remaining: 3.25s
692:	learn: 0.1104911	total: 7.31s	remaining: 3.24s
693:	learn: 0.1104771	total: 7.32s	remaining: 3.23s
694:	learn: 0.1104650	total: 7.33s	remaining: 3.22s
695:	learn: 0.1104485	total: 7.34s	remaining: 3.21s
696:	learn: 0.1104188	total: 7.35s	remaining: 3.2s
697:	learn: 0.1104145	total: 7.36s	remaining: 3.18s
698:	learn: 0.1103983	total: 7.37s	remaining: 3.17s
699:	learn: 0.1103805	total: 7.38s	remaining: 3.16s
700:	learn: 0.1103720	total: 7.39s	remaining: 3.15s
701:	learn: 0.1103428	total: 7.4s	remaining: 3.14s
702:	learn: 0.1103157	total: 7.41s	remaining: 3.13s
703:	learn: 0.1103035	total: 7.41s	remaining: 3.12s
704:	learn: 0.1102998	total: 7.42s	remaining: 3.11s
705:	learn: 0.1102753	total: 7.43s	remaining: 3.09s
706:	learn: 0.1102633	total: 7.44s	remaining: 3.08s
707:	learn: 0.1102398	total: 7.45s	remaining: 3.07s
708:	learn: 0.11

854:	learn: 0.1078586	total: 8.9s	remaining: 1.51s
855:	learn: 0.1078502	total: 8.91s	remaining: 1.5s
856:	learn: 0.1078467	total: 8.92s	remaining: 1.49s
857:	learn: 0.1078196	total: 8.93s	remaining: 1.48s
858:	learn: 0.1077947	total: 8.94s	remaining: 1.47s
859:	learn: 0.1077705	total: 8.95s	remaining: 1.46s
860:	learn: 0.1077597	total: 8.96s	remaining: 1.45s
861:	learn: 0.1077364	total: 8.97s	remaining: 1.44s
862:	learn: 0.1077250	total: 8.98s	remaining: 1.43s
863:	learn: 0.1077076	total: 8.99s	remaining: 1.41s
864:	learn: 0.1076982	total: 9s	remaining: 1.4s
865:	learn: 0.1076842	total: 9.01s	remaining: 1.39s
866:	learn: 0.1076705	total: 9.02s	remaining: 1.38s
867:	learn: 0.1076483	total: 9.03s	remaining: 1.37s
868:	learn: 0.1076380	total: 9.04s	remaining: 1.36s
869:	learn: 0.1076363	total: 9.05s	remaining: 1.35s
870:	learn: 0.1076120	total: 9.06s	remaining: 1.34s
871:	learn: 0.1075891	total: 9.07s	remaining: 1.33s
872:	learn: 0.1075809	total: 9.08s	remaining: 1.32s
873:	learn: 0.1075

In [23]:
opt_cat = CatBoostClassifier(**best_params,od_type='Iter', loss_function='Logloss', cat_features=categorical_f)
# opt_cat.fit(X_train_re, y_train_re, cat_features=categorical_f)

log_2 = cross_val_score(estimator = opt_cat, X = X_train_re, y = y_train_re, 
                             cv = 5, scoring=LogLoss)

0:	learn: 0.6865870	total: 41ms	remaining: 40.9s
1:	learn: 0.6803749	total: 79.4ms	remaining: 39.6s
2:	learn: 0.6742512	total: 119ms	remaining: 39.5s
3:	learn: 0.6683201	total: 156ms	remaining: 38.9s
4:	learn: 0.6624911	total: 190ms	remaining: 37.8s
5:	learn: 0.6568084	total: 231ms	remaining: 38.4s
6:	learn: 0.6514418	total: 255ms	remaining: 36.2s
7:	learn: 0.6460545	total: 278ms	remaining: 34.5s
8:	learn: 0.6408656	total: 316ms	remaining: 34.8s
9:	learn: 0.6357314	total: 370ms	remaining: 36.6s
10:	learn: 0.6306905	total: 433ms	remaining: 39s
11:	learn: 0.6256938	total: 461ms	remaining: 38s
12:	learn: 0.6209260	total: 500ms	remaining: 38s
13:	learn: 0.6163016	total: 541ms	remaining: 38.1s
14:	learn: 0.6118400	total: 578ms	remaining: 37.9s
15:	learn: 0.6074414	total: 614ms	remaining: 37.8s
16:	learn: 0.6031800	total: 652ms	remaining: 37.7s
17:	learn: 0.5990325	total: 693ms	remaining: 37.8s
18:	learn: 0.5948505	total: 734ms	remaining: 37.9s
19:	learn: 0.5843899	total: 778ms	remaining: 38

164:	learn: 0.1762573	total: 8s	remaining: 40.5s
165:	learn: 0.1756429	total: 8.05s	remaining: 40.5s
166:	learn: 0.1750420	total: 8.1s	remaining: 40.4s
167:	learn: 0.1744571	total: 8.15s	remaining: 40.4s
168:	learn: 0.1739060	total: 8.2s	remaining: 40.3s
169:	learn: 0.1733683	total: 8.25s	remaining: 40.3s
170:	learn: 0.1728081	total: 8.3s	remaining: 40.3s
171:	learn: 0.1722671	total: 8.36s	remaining: 40.2s
172:	learn: 0.1717442	total: 8.4s	remaining: 40.2s
173:	learn: 0.1712161	total: 8.45s	remaining: 40.1s
174:	learn: 0.1707147	total: 8.5s	remaining: 40.1s
175:	learn: 0.1701831	total: 8.56s	remaining: 40.1s
176:	learn: 0.1696047	total: 8.61s	remaining: 40s
177:	learn: 0.1691396	total: 8.67s	remaining: 40s
178:	learn: 0.1685858	total: 8.72s	remaining: 40s
179:	learn: 0.1681419	total: 8.78s	remaining: 40s
180:	learn: 0.1676683	total: 8.84s	remaining: 40s
181:	learn: 0.1671478	total: 8.89s	remaining: 40s
182:	learn: 0.1666591	total: 8.95s	remaining: 39.9s
183:	learn: 0.1662493	total: 8.9

323:	learn: 0.1428015	total: 15.7s	remaining: 32.8s
324:	learn: 0.1427936	total: 15.8s	remaining: 32.7s
325:	learn: 0.1427905	total: 15.8s	remaining: 32.7s
326:	learn: 0.1426750	total: 15.8s	remaining: 32.6s
327:	learn: 0.1426411	total: 15.9s	remaining: 32.5s
328:	learn: 0.1425512	total: 15.9s	remaining: 32.5s
329:	learn: 0.1425420	total: 16s	remaining: 32.4s
330:	learn: 0.1424841	total: 16s	remaining: 32.4s
331:	learn: 0.1424803	total: 16.1s	remaining: 32.3s
332:	learn: 0.1424710	total: 16.1s	remaining: 32.3s
333:	learn: 0.1424632	total: 16.2s	remaining: 32.2s
334:	learn: 0.1424530	total: 16.2s	remaining: 32.2s
335:	learn: 0.1424395	total: 16.2s	remaining: 32.1s
336:	learn: 0.1423285	total: 16.3s	remaining: 32s
337:	learn: 0.1423239	total: 16.3s	remaining: 32s
338:	learn: 0.1423152	total: 16.4s	remaining: 31.9s
339:	learn: 0.1423096	total: 16.4s	remaining: 31.8s
340:	learn: 0.1423010	total: 16.4s	remaining: 31.8s
341:	learn: 0.1422925	total: 16.5s	remaining: 31.7s
342:	learn: 0.142283

482:	learn: 0.1392404	total: 22.4s	remaining: 24s
483:	learn: 0.1392377	total: 22.5s	remaining: 24s
484:	learn: 0.1392259	total: 22.5s	remaining: 23.9s
485:	learn: 0.1392233	total: 22.6s	remaining: 23.9s
486:	learn: 0.1392208	total: 22.6s	remaining: 23.8s
487:	learn: 0.1392042	total: 22.6s	remaining: 23.8s
488:	learn: 0.1391829	total: 22.7s	remaining: 23.7s
489:	learn: 0.1391802	total: 22.7s	remaining: 23.7s
490:	learn: 0.1391771	total: 22.8s	remaining: 23.6s
491:	learn: 0.1391562	total: 22.8s	remaining: 23.6s
492:	learn: 0.1391541	total: 22.9s	remaining: 23.5s
493:	learn: 0.1391490	total: 22.9s	remaining: 23.5s
494:	learn: 0.1391380	total: 22.9s	remaining: 23.4s
495:	learn: 0.1391249	total: 23s	remaining: 23.4s
496:	learn: 0.1391218	total: 23s	remaining: 23.3s
497:	learn: 0.1391208	total: 23.1s	remaining: 23.3s
498:	learn: 0.1391182	total: 23.1s	remaining: 23.2s
499:	learn: 0.1391156	total: 23.1s	remaining: 23.1s
500:	learn: 0.1391124	total: 23.2s	remaining: 23.1s
501:	learn: 0.139109

642:	learn: 0.1357009	total: 29.7s	remaining: 16.5s
643:	learn: 0.1356629	total: 29.8s	remaining: 16.5s
644:	learn: 0.1356487	total: 29.8s	remaining: 16.4s
645:	learn: 0.1356071	total: 29.9s	remaining: 16.4s
646:	learn: 0.1355975	total: 29.9s	remaining: 16.3s
647:	learn: 0.1355902	total: 30s	remaining: 16.3s
648:	learn: 0.1355721	total: 30s	remaining: 16.2s
649:	learn: 0.1355465	total: 30.1s	remaining: 16.2s
650:	learn: 0.1355230	total: 30.1s	remaining: 16.1s
651:	learn: 0.1355143	total: 30.2s	remaining: 16.1s
652:	learn: 0.1354926	total: 30.2s	remaining: 16s
653:	learn: 0.1354661	total: 30.2s	remaining: 16s
654:	learn: 0.1354260	total: 30.3s	remaining: 16s
655:	learn: 0.1354073	total: 30.3s	remaining: 15.9s
656:	learn: 0.1354033	total: 30.4s	remaining: 15.9s
657:	learn: 0.1353662	total: 30.4s	remaining: 15.8s
658:	learn: 0.1353599	total: 30.5s	remaining: 15.8s
659:	learn: 0.1353351	total: 30.5s	remaining: 15.7s
660:	learn: 0.1353000	total: 30.6s	remaining: 15.7s
661:	learn: 0.1352886	

...

352:	learn: 0.1338843	total: 18.3s	remaining: 33.5s
353:	learn: 0.1338353	total: 18.3s	remaining: 33.5s
354:	learn: 0.1337854	total: 18.4s	remaining: 33.4s
355:	learn: 0.1337400	total: 18.4s	remaining: 33.4s
356:	learn: 0.1336773	total: 18.5s	remaining: 33.3s
357:	learn: 0.1336449	total: 18.6s	remaining: 33.3s
358:	learn: 0.1336303	total: 18.6s	remaining: 33.2s
359:	learn: 0.1335857	total: 18.7s	remaining: 33.2s
360:	learn: 0.1335732	total: 18.7s	remaining: 33.1s
361:	learn: 0.1335591	total: 18.8s	remaining: 33.1s
362:	learn: 0.1334814	total: 18.8s	remaining: 33s
363:	learn: 0.1334224	total: 18.9s	remaining: 33s
364:	learn: 0.1334006	total: 18.9s	remaining: 32.9s
365:	learn: 0.1333859	total: 19s	remaining: 32.9s
366:	learn: 0.1333438	total: 19s	remaining: 32.8s
367:	learn: 0.1333265	total: 19.1s	remaining: 32.7s
368:	learn: 0.1333071	total: 19.1s	remaining: 32.7s
369:	learn: 0.1332653	total: 19.2s	remaining: 32.6s
370:	learn: 0.1332505	total: 19.2s	remaining: 32.6s
371:	learn: 0.133245

512:	learn: 0.1298283	total: 26.1s	remaining: 24.8s
513:	learn: 0.1298066	total: 26.1s	remaining: 24.7s
514:	learn: 0.1297847	total: 26.2s	remaining: 24.6s
515:	learn: 0.1297602	total: 26.2s	remaining: 24.6s
516:	learn: 0.1297388	total: 26.3s	remaining: 24.5s
517:	learn: 0.1297353	total: 26.3s	remaining: 24.5s
518:	learn: 0.1297285	total: 26.3s	remaining: 24.4s
519:	learn: 0.1297240	total: 26.4s	remaining: 24.4s
520:	learn: 0.1296521	total: 26.4s	remaining: 24.3s
521:	learn: 0.1296359	total: 26.5s	remaining: 24.3s
522:	learn: 0.1295836	total: 26.6s	remaining: 24.2s
523:	learn: 0.1295784	total: 26.6s	remaining: 24.2s
524:	learn: 0.1295582	total: 26.6s	remaining: 24.1s
525:	learn: 0.1295553	total: 26.7s	remaining: 24.1s
526:	learn: 0.1295473	total: 26.7s	remaining: 24s
527:	learn: 0.1295395	total: 26.8s	remaining: 23.9s
528:	learn: 0.1295214	total: 26.8s	remaining: 23.9s
529:	learn: 0.1295036	total: 26.9s	remaining: 23.8s
530:	learn: 0.1294977	total: 26.9s	remaining: 23.8s
531:	learn: 0.

671:	learn: 0.1266665	total: 33.6s	remaining: 16.4s
672:	learn: 0.1266623	total: 33.6s	remaining: 16.3s
673:	learn: 0.1266260	total: 33.7s	remaining: 16.3s
674:	learn: 0.1266026	total: 33.7s	remaining: 16.2s
675:	learn: 0.1265978	total: 33.8s	remaining: 16.2s
676:	learn: 0.1265687	total: 33.8s	remaining: 16.1s
677:	learn: 0.1265492	total: 33.9s	remaining: 16.1s
678:	learn: 0.1265432	total: 33.9s	remaining: 16s
679:	learn: 0.1265392	total: 34s	remaining: 16s
680:	learn: 0.1265353	total: 34s	remaining: 15.9s
681:	learn: 0.1264996	total: 34.1s	remaining: 15.9s
682:	learn: 0.1264686	total: 34.1s	remaining: 15.8s
683:	learn: 0.1264479	total: 34.2s	remaining: 15.8s
684:	learn: 0.1264283	total: 34.2s	remaining: 15.7s
685:	learn: 0.1264224	total: 34.3s	remaining: 15.7s
686:	learn: 0.1264062	total: 34.3s	remaining: 15.6s
687:	learn: 0.1263918	total: 34.3s	remaining: 15.6s
688:	learn: 0.1263856	total: 34.4s	remaining: 15.5s
689:	learn: 0.1263819	total: 34.4s	remaining: 15.5s
690:	learn: 0.126351

831:	learn: 0.1233389	total: 41.5s	remaining: 8.37s
832:	learn: 0.1233312	total: 41.5s	remaining: 8.32s
833:	learn: 0.1233267	total: 41.6s	remaining: 8.27s
834:	learn: 0.1233019	total: 41.6s	remaining: 8.22s
835:	learn: 0.1232607	total: 41.7s	remaining: 8.18s
836:	learn: 0.1232551	total: 41.7s	remaining: 8.13s
837:	learn: 0.1232184	total: 41.8s	remaining: 8.08s
838:	learn: 0.1232141	total: 41.8s	remaining: 8.03s
839:	learn: 0.1232108	total: 41.9s	remaining: 7.97s
840:	learn: 0.1231577	total: 41.9s	remaining: 7.93s
841:	learn: 0.1231450	total: 42s	remaining: 7.88s
842:	learn: 0.1231412	total: 42s	remaining: 7.83s
843:	learn: 0.1231118	total: 42.1s	remaining: 7.77s
844:	learn: 0.1230682	total: 42.1s	remaining: 7.72s
845:	learn: 0.1230361	total: 42.2s	remaining: 7.68s
846:	learn: 0.1229931	total: 42.2s	remaining: 7.63s
847:	learn: 0.1229916	total: 42.3s	remaining: 7.58s
848:	learn: 0.1229563	total: 42.3s	remaining: 7.53s
849:	learn: 0.1229517	total: 42.4s	remaining: 7.48s
850:	learn: 0.12

990:	learn: 0.1202613	total: 49.5s	remaining: 449ms
991:	learn: 0.1202158	total: 49.5s	remaining: 399ms
992:	learn: 0.1201917	total: 49.6s	remaining: 350ms
993:	learn: 0.1201823	total: 49.6s	remaining: 300ms
994:	learn: 0.1201734	total: 49.7s	remaining: 250ms
995:	learn: 0.1201687	total: 49.8s	remaining: 200ms
996:	learn: 0.1201664	total: 49.8s	remaining: 150ms
997:	learn: 0.1201365	total: 49.8s	remaining: 99.9ms
998:	learn: 0.1201269	total: 49.9s	remaining: 49.9ms
999:	learn: 0.1201181	total: 49.9s	remaining: 0us


In [24]:
print("Log loss: {:.3f}".format(log_2.mean()))

Log loss: -0.189


In [29]:
opt_cat = CatBoostClassifier(**best_params,od_type='Iter', loss_function='Logloss', cat_features=categorical_f)
opt_cat.fit(X_train, y_train)
y_pred_cat = opt_cat.predict_proba(X_test.values)[:, 1]
print("model logloss: %.3f" % log_loss(y_test, y_pred_cat))

0:	learn: 0.6875355	total: 42.4ms	remaining: 42.3s
1:	learn: 0.6820192	total: 79.5ms	remaining: 39.7s
2:	learn: 0.6766395	total: 116ms	remaining: 38.5s
3:	learn: 0.6713778	total: 151ms	remaining: 37.7s
4:	learn: 0.6663018	total: 186ms	remaining: 37s
5:	learn: 0.6612979	total: 221ms	remaining: 36.5s
6:	learn: 0.6563950	total: 258ms	remaining: 36.7s
7:	learn: 0.6516036	total: 303ms	remaining: 37.6s
8:	learn: 0.6469422	total: 339ms	remaining: 37.3s
9:	learn: 0.6423904	total: 370ms	remaining: 36.7s
10:	learn: 0.6380832	total: 404ms	remaining: 36.4s
11:	learn: 0.6337515	total: 437ms	remaining: 36s
12:	learn: 0.6295525	total: 477ms	remaining: 36.2s
13:	learn: 0.6255165	total: 514ms	remaining: 36.2s
14:	learn: 0.6214427	total: 548ms	remaining: 36s
15:	learn: 0.6176111	total: 586ms	remaining: 36s
16:	learn: 0.6137627	total: 624ms	remaining: 36.1s
17:	learn: 0.6099900	total: 661ms	remaining: 36.1s
18:	learn: 0.6062864	total: 701ms	remaining: 36.2s
19:	learn: 0.6026857	total: 734ms	remaining: 36

164:	learn: 0.4437489	total: 5.97s	remaining: 30.2s
165:	learn: 0.4434903	total: 6s	remaining: 30.2s
166:	learn: 0.4432688	total: 6.04s	remaining: 30.1s
167:	learn: 0.4430163	total: 6.08s	remaining: 30.1s
168:	learn: 0.4427748	total: 6.12s	remaining: 30.1s
169:	learn: 0.4425091	total: 6.16s	remaining: 30.1s
170:	learn: 0.4422441	total: 6.2s	remaining: 30.1s
171:	learn: 0.4420073	total: 6.24s	remaining: 30s
172:	learn: 0.4417742	total: 6.28s	remaining: 30s
173:	learn: 0.4415160	total: 6.32s	remaining: 30s
174:	learn: 0.4412894	total: 6.36s	remaining: 30s
175:	learn: 0.4410826	total: 6.4s	remaining: 30s
176:	learn: 0.4408805	total: 6.45s	remaining: 30s
177:	learn: 0.4406627	total: 6.49s	remaining: 30s
178:	learn: 0.4404272	total: 6.54s	remaining: 30s
179:	learn: 0.4402318	total: 6.58s	remaining: 30s
180:	learn: 0.4400512	total: 6.62s	remaining: 29.9s
181:	learn: 0.4398593	total: 6.66s	remaining: 29.9s
182:	learn: 0.4397274	total: 6.69s	remaining: 29.9s
183:	learn: 0.4395190	total: 6.73s	

327:	learn: 0.4207958	total: 14.2s	remaining: 29.1s
328:	learn: 0.4207310	total: 14.3s	remaining: 29.1s
329:	learn: 0.4206688	total: 14.3s	remaining: 29s
330:	learn: 0.4205811	total: 14.4s	remaining: 29s
331:	learn: 0.4205284	total: 14.4s	remaining: 29s
332:	learn: 0.4204880	total: 14.4s	remaining: 28.9s
333:	learn: 0.4204222	total: 14.5s	remaining: 28.9s
334:	learn: 0.4203501	total: 14.5s	remaining: 28.8s
335:	learn: 0.4203215	total: 14.6s	remaining: 28.8s
336:	learn: 0.4202611	total: 14.6s	remaining: 28.7s
337:	learn: 0.4201617	total: 14.7s	remaining: 28.7s
338:	learn: 0.4200692	total: 14.7s	remaining: 28.7s
339:	learn: 0.4200111	total: 14.8s	remaining: 28.7s
340:	learn: 0.4199949	total: 14.8s	remaining: 28.7s
341:	learn: 0.4199446	total: 14.9s	remaining: 28.6s
342:	learn: 0.4198985	total: 15s	remaining: 28.6s
343:	learn: 0.4198607	total: 15s	remaining: 28.6s
344:	learn: 0.4197896	total: 15s	remaining: 28.6s
345:	learn: 0.4197071	total: 15.1s	remaining: 28.5s
346:	learn: 0.4196759	to

487:	learn: 0.4127613	total: 21.6s	remaining: 22.7s
488:	learn: 0.4126987	total: 21.7s	remaining: 22.7s
489:	learn: 0.4126566	total: 21.8s	remaining: 22.6s
490:	learn: 0.4126144	total: 21.8s	remaining: 22.6s
491:	learn: 0.4125881	total: 21.8s	remaining: 22.6s
492:	learn: 0.4125562	total: 21.9s	remaining: 22.5s
493:	learn: 0.4125506	total: 21.9s	remaining: 22.5s
494:	learn: 0.4125357	total: 22s	remaining: 22.4s
495:	learn: 0.4124795	total: 22s	remaining: 22.4s
496:	learn: 0.4124435	total: 22.1s	remaining: 22.3s
497:	learn: 0.4124245	total: 22.1s	remaining: 22.3s
498:	learn: 0.4123689	total: 22.1s	remaining: 22.2s
499:	learn: 0.4123543	total: 22.2s	remaining: 22.2s
500:	learn: 0.4123151	total: 22.2s	remaining: 22.1s
501:	learn: 0.4122444	total: 22.3s	remaining: 22.1s
502:	learn: 0.4121703	total: 22.3s	remaining: 22.1s
503:	learn: 0.4121485	total: 22.4s	remaining: 22s
504:	learn: 0.4121406	total: 22.4s	remaining: 22s
505:	learn: 0.4121173	total: 22.4s	remaining: 21.9s
506:	learn: 0.412079

649:	learn: 0.4070125	total: 29.1s	remaining: 15.7s
650:	learn: 0.4069887	total: 29.2s	remaining: 15.6s
651:	learn: 0.4069835	total: 29.2s	remaining: 15.6s
652:	learn: 0.4069669	total: 29.3s	remaining: 15.6s
653:	learn: 0.4069347	total: 29.3s	remaining: 15.5s
654:	learn: 0.4068804	total: 29.4s	remaining: 15.5s
655:	learn: 0.4068243	total: 29.4s	remaining: 15.4s
656:	learn: 0.4067819	total: 29.4s	remaining: 15.4s
657:	learn: 0.4067509	total: 29.5s	remaining: 15.3s
658:	learn: 0.4067164	total: 29.5s	remaining: 15.3s
659:	learn: 0.4067101	total: 29.6s	remaining: 15.2s
660:	learn: 0.4067056	total: 29.6s	remaining: 15.2s
661:	learn: 0.4066742	total: 29.6s	remaining: 15.1s
662:	learn: 0.4066515	total: 29.7s	remaining: 15.1s
663:	learn: 0.4066096	total: 29.7s	remaining: 15s
664:	learn: 0.4065788	total: 29.8s	remaining: 15s
665:	learn: 0.4065269	total: 29.8s	remaining: 14.9s
666:	learn: 0.4064997	total: 29.8s	remaining: 14.9s
667:	learn: 0.4064699	total: 29.9s	remaining: 14.9s
668:	learn: 0.40

808:	learn: 0.4018779	total: 36s	remaining: 8.49s
809:	learn: 0.4018330	total: 36s	remaining: 8.45s
810:	learn: 0.4018032	total: 36.1s	remaining: 8.4s
811:	learn: 0.4017798	total: 36.1s	remaining: 8.36s
812:	learn: 0.4017712	total: 36.2s	remaining: 8.32s
813:	learn: 0.4017462	total: 36.2s	remaining: 8.27s
814:	learn: 0.4017196	total: 36.2s	remaining: 8.23s
815:	learn: 0.4017089	total: 36.3s	remaining: 8.19s
816:	learn: 0.4016881	total: 36.4s	remaining: 8.14s
817:	learn: 0.4016757	total: 36.4s	remaining: 8.1s
818:	learn: 0.4016283	total: 36.5s	remaining: 8.06s
819:	learn: 0.4016049	total: 36.5s	remaining: 8.01s
820:	learn: 0.4015694	total: 36.5s	remaining: 7.97s
821:	learn: 0.4015548	total: 36.6s	remaining: 7.92s
822:	learn: 0.4015351	total: 36.6s	remaining: 7.88s
823:	learn: 0.4015080	total: 36.7s	remaining: 7.83s
824:	learn: 0.4014987	total: 36.7s	remaining: 7.79s
825:	learn: 0.4014864	total: 36.8s	remaining: 7.74s
826:	learn: 0.4014485	total: 36.8s	remaining: 7.7s
827:	learn: 0.40142

970:	learn: 0.3969013	total: 43.4s	remaining: 1.3s
971:	learn: 0.3968690	total: 43.5s	remaining: 1.25s
972:	learn: 0.3968457	total: 43.6s	remaining: 1.21s
973:	learn: 0.3968159	total: 43.6s	remaining: 1.16s
974:	learn: 0.3967491	total: 43.6s	remaining: 1.12s
975:	learn: 0.3967206	total: 43.7s	remaining: 1.07s
976:	learn: 0.3966734	total: 43.8s	remaining: 1.03s
977:	learn: 0.3966318	total: 43.8s	remaining: 986ms
978:	learn: 0.3965987	total: 43.9s	remaining: 941ms
979:	learn: 0.3965910	total: 43.9s	remaining: 896ms
980:	learn: 0.3965830	total: 44s	remaining: 852ms
981:	learn: 0.3965233	total: 44s	remaining: 807ms
982:	learn: 0.3964376	total: 44.1s	remaining: 762ms
983:	learn: 0.3963912	total: 44.1s	remaining: 718ms
984:	learn: 0.3963537	total: 44.2s	remaining: 673ms
985:	learn: 0.3963371	total: 44.2s	remaining: 628ms
986:	learn: 0.3962971	total: 44.3s	remaining: 583ms
987:	learn: 0.3962895	total: 44.3s	remaining: 538ms
988:	learn: 0.3962592	total: 44.3s	remaining: 493ms
989:	learn: 0.396

In [ ]:
def random_search(param_grid, out_file, max_evals = MAX_EVALS):
    """Random search for hyperparameter optimization. 
       Writes result of search to csv file every search iteration."""
    
    
    # Dataframe for results
    results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                                  index = list(range(MAX_EVALS)))
    for i in range(MAX_EVALS):
        
        # Choose random hyperparameters
        random_params = {k: random.sample(v, 1)[0] for k, v in param_grid.items()}
        random_params['subsample'] = 1.0 if random_params['boosting_type'] == 'goss' else random_params['subsample']

        # Evaluate randomly selected hyperparameters
        eval_results = objective(random_params, i)
        results.loc[i, :] = eval_results

        # open connection (append option) and write results
        of_connection = open(out_file, 'a')
        writer = csv.writer(of_connection)
        writer.writerow(eval_results)
        
        # make sure to close connection
        of_connection.close()
        
    # Sort with best score on top
    results.sort_values('score', ascending = False, inplace = True)
    results.reset_index(inplace = True)

    return results 

### Output_1

In [33]:
opt_cat = CatBoostClassifier(**best_params,od_type='Iter', loss_function='Logloss', cat_features=categorical_f)
opt_cat.fit(df_copy, Y)

0:	learn: 0.6876016	total: 37.8ms	remaining: 37.8s
1:	learn: 0.6821458	total: 77.9ms	remaining: 38.9s
2:	learn: 0.6768222	total: 114ms	remaining: 38s
3:	learn: 0.6716256	total: 150ms	remaining: 37.4s
4:	learn: 0.6665317	total: 186ms	remaining: 37s
5:	learn: 0.6615421	total: 224ms	remaining: 37.1s
6:	learn: 0.6567268	total: 270ms	remaining: 38.3s
7:	learn: 0.6520342	total: 309ms	remaining: 38.4s
8:	learn: 0.6474125	total: 345ms	remaining: 38s
9:	learn: 0.6428989	total: 381ms	remaining: 37.7s
10:	learn: 0.6384977	total: 413ms	remaining: 37.1s
11:	learn: 0.6341903	total: 446ms	remaining: 36.8s
12:	learn: 0.6299779	total: 482ms	remaining: 36.6s
13:	learn: 0.6258756	total: 523ms	remaining: 36.8s
14:	learn: 0.6218983	total: 560ms	remaining: 36.8s
15:	learn: 0.6179753	total: 599ms	remaining: 36.8s
16:	learn: 0.6142200	total: 633ms	remaining: 36.6s
17:	learn: 0.6104715	total: 666ms	remaining: 36.3s
18:	learn: 0.6068361	total: 707ms	remaining: 36.5s
19:	learn: 0.6032959	total: 754ms	remaining: 

163:	learn: 0.4467688	total: 7.01s	remaining: 35.7s
164:	learn: 0.4465398	total: 7.06s	remaining: 35.7s
165:	learn: 0.4463409	total: 7.11s	remaining: 35.7s
166:	learn: 0.4461307	total: 7.16s	remaining: 35.7s
167:	learn: 0.4459211	total: 7.2s	remaining: 35.7s
168:	learn: 0.4457363	total: 7.25s	remaining: 35.7s
169:	learn: 0.4455335	total: 7.31s	remaining: 35.7s
170:	learn: 0.4453196	total: 7.36s	remaining: 35.7s
171:	learn: 0.4451401	total: 7.41s	remaining: 35.7s
172:	learn: 0.4449336	total: 7.46s	remaining: 35.7s
173:	learn: 0.4447397	total: 7.51s	remaining: 35.6s
174:	learn: 0.4446155	total: 7.55s	remaining: 35.6s
175:	learn: 0.4444641	total: 7.6s	remaining: 35.6s
176:	learn: 0.4443120	total: 7.65s	remaining: 35.6s
177:	learn: 0.4441577	total: 7.72s	remaining: 35.7s
178:	learn: 0.4440145	total: 7.79s	remaining: 35.7s
179:	learn: 0.4438734	total: 7.85s	remaining: 35.8s
180:	learn: 0.4436916	total: 7.9s	remaining: 35.8s
181:	learn: 0.4435609	total: 7.95s	remaining: 35.7s
182:	learn: 0.4

325:	learn: 0.4251704	total: 15.4s	remaining: 31.8s
326:	learn: 0.4250799	total: 15.4s	remaining: 31.8s
327:	learn: 0.4249846	total: 15.5s	remaining: 31.7s
328:	learn: 0.4249004	total: 15.6s	remaining: 31.7s
329:	learn: 0.4248181	total: 15.6s	remaining: 31.7s
330:	learn: 0.4247516	total: 15.7s	remaining: 31.7s
331:	learn: 0.4246609	total: 15.8s	remaining: 31.7s
332:	learn: 0.4245710	total: 15.8s	remaining: 31.7s
333:	learn: 0.4245002	total: 15.9s	remaining: 31.7s
334:	learn: 0.4243986	total: 15.9s	remaining: 31.6s
335:	learn: 0.4242974	total: 16s	remaining: 31.6s
336:	learn: 0.4242170	total: 16.1s	remaining: 31.6s
337:	learn: 0.4241757	total: 16.1s	remaining: 31.6s
338:	learn: 0.4240839	total: 16.2s	remaining: 31.6s
339:	learn: 0.4240379	total: 16.2s	remaining: 31.5s
340:	learn: 0.4239635	total: 16.3s	remaining: 31.5s
341:	learn: 0.4239126	total: 16.4s	remaining: 31.5s
342:	learn: 0.4238606	total: 16.4s	remaining: 31.4s
343:	learn: 0.4237977	total: 16.5s	remaining: 31.4s
344:	learn: 0.

484:	learn: 0.4183183	total: 24.6s	remaining: 26.1s
485:	learn: 0.4182884	total: 24.7s	remaining: 26.1s
486:	learn: 0.4182569	total: 24.7s	remaining: 26s
487:	learn: 0.4182362	total: 24.8s	remaining: 26s
488:	learn: 0.4182184	total: 24.8s	remaining: 25.9s
489:	learn: 0.4181925	total: 24.9s	remaining: 25.9s
490:	learn: 0.4181637	total: 24.9s	remaining: 25.8s
491:	learn: 0.4181346	total: 25s	remaining: 25.8s
492:	learn: 0.4181060	total: 25s	remaining: 25.7s
493:	learn: 0.4180996	total: 25.1s	remaining: 25.7s
494:	learn: 0.4180321	total: 25.1s	remaining: 25.6s
495:	learn: 0.4180217	total: 25.2s	remaining: 25.6s
496:	learn: 0.4179978	total: 25.2s	remaining: 25.5s
497:	learn: 0.4179654	total: 25.3s	remaining: 25.5s
498:	learn: 0.4179464	total: 25.3s	remaining: 25.4s
499:	learn: 0.4178968	total: 25.4s	remaining: 25.4s
500:	learn: 0.4178813	total: 25.4s	remaining: 25.3s
501:	learn: 0.4178577	total: 25.5s	remaining: 25.3s
502:	learn: 0.4178389	total: 25.5s	remaining: 25.2s
503:	learn: 0.417817

643:	learn: 0.4143940	total: 33.2s	remaining: 18.3s
644:	learn: 0.4143369	total: 33.2s	remaining: 18.3s
645:	learn: 0.4143307	total: 33.3s	remaining: 18.2s
646:	learn: 0.4142956	total: 33.3s	remaining: 18.2s
647:	learn: 0.4142874	total: 33.4s	remaining: 18.1s
648:	learn: 0.4142300	total: 33.4s	remaining: 18.1s
649:	learn: 0.4142098	total: 33.5s	remaining: 18s
650:	learn: 0.4142018	total: 33.5s	remaining: 18s
651:	learn: 0.4141751	total: 33.6s	remaining: 17.9s
652:	learn: 0.4141712	total: 33.6s	remaining: 17.9s
653:	learn: 0.4141331	total: 33.7s	remaining: 17.8s
654:	learn: 0.4140896	total: 33.7s	remaining: 17.8s
655:	learn: 0.4140503	total: 33.8s	remaining: 17.7s
656:	learn: 0.4140323	total: 33.8s	remaining: 17.7s
657:	learn: 0.4140286	total: 33.9s	remaining: 17.6s
658:	learn: 0.4140165	total: 34s	remaining: 17.6s
659:	learn: 0.4140035	total: 34s	remaining: 17.5s
660:	learn: 0.4139874	total: 34.1s	remaining: 17.5s
661:	learn: 0.4139668	total: 34.1s	remaining: 17.4s
662:	learn: 0.413924

802:	learn: 0.4103689	total: 41.7s	remaining: 10.2s
803:	learn: 0.4103639	total: 41.7s	remaining: 10.2s
804:	learn: 0.4103431	total: 41.8s	remaining: 10.1s
805:	learn: 0.4102792	total: 41.8s	remaining: 10.1s
806:	learn: 0.4102618	total: 41.9s	remaining: 10s
807:	learn: 0.4102476	total: 41.9s	remaining: 9.96s
808:	learn: 0.4101754	total: 42s	remaining: 9.92s
809:	learn: 0.4101689	total: 42.1s	remaining: 9.86s
810:	learn: 0.4101160	total: 42.1s	remaining: 9.81s
811:	learn: 0.4100946	total: 42.2s	remaining: 9.76s
812:	learn: 0.4100718	total: 42.2s	remaining: 9.71s
813:	learn: 0.4100307	total: 42.3s	remaining: 9.66s
814:	learn: 0.4099833	total: 42.3s	remaining: 9.61s
815:	learn: 0.4099638	total: 42.4s	remaining: 9.56s
816:	learn: 0.4099486	total: 42.4s	remaining: 9.51s
817:	learn: 0.4099373	total: 42.5s	remaining: 9.45s
818:	learn: 0.4099139	total: 42.5s	remaining: 9.4s
819:	learn: 0.4098590	total: 42.6s	remaining: 9.35s
820:	learn: 0.4098335	total: 42.7s	remaining: 9.3s
821:	learn: 0.4098

961:	learn: 0.4061496	total: 51.9s	remaining: 2.05s
962:	learn: 0.4061288	total: 52s	remaining: 2s
963:	learn: 0.4060975	total: 52s	remaining: 1.94s
964:	learn: 0.4060600	total: 52.1s	remaining: 1.89s
965:	learn: 0.4060170	total: 52.2s	remaining: 1.84s
966:	learn: 0.4059948	total: 52.2s	remaining: 1.78s
967:	learn: 0.4059736	total: 52.3s	remaining: 1.73s
968:	learn: 0.4059436	total: 52.3s	remaining: 1.67s
969:	learn: 0.4059073	total: 52.4s	remaining: 1.62s
970:	learn: 0.4058730	total: 52.5s	remaining: 1.57s
971:	learn: 0.4058516	total: 52.6s	remaining: 1.51s
972:	learn: 0.4058463	total: 52.6s	remaining: 1.46s
973:	learn: 0.4058428	total: 52.7s	remaining: 1.41s
974:	learn: 0.4058229	total: 52.8s	remaining: 1.35s
975:	learn: 0.4058096	total: 52.8s	remaining: 1.3s
976:	learn: 0.4057693	total: 52.9s	remaining: 1.25s
977:	learn: 0.4057553	total: 52.9s	remaining: 1.19s
978:	learn: 0.4057544	total: 53s	remaining: 1.14s
979:	learn: 0.4057393	total: 53.1s	remaining: 1.08s
980:	learn: 0.4057099	

In [35]:
df_test = pd.read_csv("test.gz", compression='gzip', header='infer')
df_test_copy = df_test.copy()
unused_cols = ['id','site_id', 'app_id']
df_test_copy.drop(unused_cols, axis=1, inplace=True)
df_test_copy.drop(['hour', 'app_domain', 'app_category'], axis=1,inplace=True)

ctr_pred = opt_cat.predict_proba(df_test_copy)[:, 1]
all_id = df_test['id']
df_out = pd.DataFrame({'id': all_id, 'ctr': ctr_pred})
df_out.to_csv('Submission.csv', index=False)

### Logistic Regression (provided)

In [16]:
# embedding (all features are categorical)
print('embedding...')
from sklearn.feature_extraction import DictVectorizer
import pickle

try:
    with open('X_train_dict.pkl', 'rb') as ff:
        X_train_dict = pickle.load(ff)
    vectorizer = joblib.load('vectorizer.joblib')
    X_train = vectorizer.transform(X_train_dict)
    print('saved vectorizer loaded & applied to training set')
except:
    X_train_dict = list(df_copy.drop('click', axis=1).T.to_dict().values())
    with open('X_train_dict.pkl', 'wb') as ff:
        pickle.dump(X_train_dict, ff)
    vectorizer = DictVectorizer(sparse=True)
    X_train = vectorizer.fit_transform(X_train_dict) # can only see training dataset
    joblib.dump(vectorizer, 'vectorizer.joblib')
    print('imported data & built a vectorizer on the training set')

n, d = X_train.shape
print("n = {}, d = {}".format(n, d))

embedding...
imported data & built a vectorizer on the training set
n = 20000, d = 14076


In [17]:
X_train

<20000x14076 sparse matrix of type '<class 'numpy.float64'>'
	with 340000 stored elements in Compressed Sparse Row format>

In [18]:
# train
from sklearn.linear_model import LogisticRegression
print('fit a simple logistic regression with l1 regularization...')
clf = LogisticRegression(max_iter=20000, penalty='l1', solver='liblinear', C=1)
clf.fit(X_train, y_train)
print('...done training')

fit a simple logistic regression with l1 regularization...
...done training


In [20]:
##########################################################
######### testing code ###################################

# transform test data as well
print('loading and transforming test data...')
df_test = pd.read_csv("test.gz", compression='gzip', header='infer')
# df_test.set_index('id', inplace=True)
unused_cols = ['id', 'site_id', 'app_id']
df_test.drop(unused_cols, axis=1, inplace=True)

try:
    with open('X_test_dict.pkl', 'rb') as ff:
        X_test_dict = pickle.load('ff')
except:
    X_test_dict = list(df_test.T.to_dict().values())
    with open('X_test_dict.pkl', 'wb') as ff:
        pickle.dump(X_test_dict, ff)

X_test = vectorizer.transform(X_test_dict)

loading and transforming test data...


In [21]:
print('predicting and output to csv...')
ctr_pred = clf.predict_proba(X_test)[:, 1]
# save output: every line is (id, ctr_pred)
all_id = df_test['id']
df_out = pd.DataFrame({'id': all_id, 'ctr': ctr_pred})
df_out.to_csv('Submission.csv', index=False)

print('...done')

predicting and output to csv...
...done
